# Setup

In [28]:
# was needed to download files from repo

# Download TorchVision repo to use some files from
# references/detection
# !git clone https://github.com/pytorch/vision.git
# !cd vision
# !git checkout v0.3.0

# !cp references/detection/utils.py ../
# !cp references/detection/transforms.py ../
# !cp references/detection/coco_eval.py ../
# !cp references/detection/engine.py ../
# !cp references/detection/coco_utils.py ../

# Imports

In [29]:
import fiftyone as fo
import fiftyone.utils.coco as fouc
from fiftyone.core.labels import Detection
from PIL import Image

import torch
import torch.nn as nn
import torch.utils
import torch.utils.data

import torchvision
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
from torchvision.models.detection.mask_rcnn import maskrcnn_resnet50_fpn, MaskRCNNPredictor
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import matplotlib.pyplot as plt
import numpy as np

import os
import importlib
from pathlib import Path

import utils
import train
from train import train_one_epoch, evaluate
import transforms as T
import ENEE439.Capstone.models.DatasetLoaders as DatasetLoaders
importlib.reload(DatasetLoaders)
from ENEE439.Capstone.models.DatasetLoaders import HRSIDSegmentationDataset


In [30]:
importlib.reload(utils)
importlib.reload(T)
importlib.reload(train)

<module 'train' from '/home/k3vinli/ENEE439/Capstone/models/mask_rcnn/train.py'>

# loading data into fiftyone

In [31]:
p = Path()
top_dir = p.absolute().parents[1]
top_dir

PosixPath('/home/k3vinli/ENEE439/Capstone')

In [32]:
# Loading Dataset
name = "HRSID"
if name in fo.list_datasets():
    dataset_traintest = fo.load_dataset(name)
else:
    dataset_dir = top_dir / "Datasets" / "HRSID"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_traintest = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=dataset_type,
        name=name,
    )

In [33]:
# dataset_traintest.delete()

In [34]:
name = "HRSID_train"
if name in fo.list_datasets():
    dataset_train = fo.load_dataset(name)
else:
    dataset_dir = top_dir / "Datasets" / "HRSID"
    label_path = top_dir / "Datasets"/ "HRSID" / "annotations" / "train2017.json"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_train = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=dataset_type,
        name=name,
        labels_path=label_path
    )

In [35]:
# dataset_train.delete()

In [36]:
name = "HRSID_test"
if name in fo.list_datasets():
    dataset_test = fo.load_dataset(name)
else:
    dataset_dir = top_dir / "Datasets" / "HRSID"
    label_path = top_dir / "Datasets"/ "HRSID" / "annotations" / "test2017.json"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_test = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=dataset_type,
        name=name,
        labels_path=label_path
    )

In [37]:
dataset_train.compute_metadata()
dataset_test.compute_metadata()

# explore fiftyone useage

In [38]:
paths = dataset_train.values("filepath")
sample = dataset_train[paths[1]]
sample.metadata

<ImageMetadata: {
    'size_bytes': None,
    'mime_type': None,
    'width': 800,
    'height': 800,
    'num_channels': None,
}>

In [39]:
#sample

In [40]:
img = Image.open(paths[1]).convert("RGB")
detections = sample["detections"].detections
segmentations = sample["segmentations"].detections

In [41]:
classes = dataset_train.distinct("%s.detections.label" % "segmentations")

In [42]:
# segmentations

In [43]:
labels_map_rev = {c: i for i, c in enumerate(classes)}
labels_map_rev


{'ship': 0}

In [44]:
for det in segmentations:
    category_id=labels_map_rev[det.label]
    coco_obj=fouc.COCOObject.from_label(det, sample.metadata, category_id=category_id)
    x,y,w,h=coco_obj.bbox

In [45]:
# [coco_obj.segmentation], dtype=torch.float

In [46]:
sample = dataset_test.first()
frame_size = (sample.metadata["width"], sample.metadata["height"])
detection = sample["segmentations"]["detections"][0]

segmentation = detection.to_segmentation(frame_size=frame_size)
full_img_mask = segmentation.mask
print(type(detection))
print("frame size", frame_size)
print("detection:", detection)
print("segmentation:", segmentation)
print("full img", full_img_mask.max())

<class 'fiftyone.core.labels.Detection'>
frame size (800, 800)
detection: <Detection: {
    'id': '645526021951373c4285f5de',
    'attributes': {},
    'tags': [],
    'label': 'ship',
    'bounding_box': [0.29625, 0.09875, 0.0525, 0.01],
    'mask': array([[False, False, False, False, False, False, False, False, False,
            False, False, False, False, False, False, False,  True,  True,
             True,  True,  True,  True,  True,  True,  True,  True,  True,
             True,  True,  True,  True,  True,  True,  True,  True,  True,
             True,  True, False, False, False, False],
           [False,  True,  True,  True,  True,  True,  True,  True,  True,
             True,  True,  True,  True,  True,  True,  True,  True,  True,
             True,  True,  True,  True,  True,  True,  True,  True,  True,
             True,  True,  True,  True,  True,  True,  True,  True,  True,
             True,  True,  True,  True,  True, False],
           [ True,  True,  True,  True,  Tr

In [47]:
session = fo.launch_app()

# Training ResNet Model

In [48]:
def get_model(num_classes, pretrained=True):
    if pretrained:
        weights = MaskRCNN_ResNet50_FPN_Weights.DEFAULT
    else:
        weights = None
    model = maskrcnn_resnet50_fpn(weights=weights)

    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [49]:
def get_transform():
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    
    return T.Compose(transforms)

In [58]:
def do_training(model, torch_dataset, torch_dataset_test, num_epochs=4, print_freq=10):
    data_loader = torch.utils.data.DataLoader(
        torch_dataset, batch_size=4, shuffle=False, num_workers=8,
        collate_fn=utils.collate_fn
    )
    data_loader_test = torch.utils.data.DataLoader(
        torch_dataset_test, batch_size=2, shuffle=False, num_workers=8,
        collate_fn=utils.collate_fn
    )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using {device}")
    model.to(device)

    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.001,
                                momentum=0.5, weight_decay=0.0005)

    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
                                                    step_size=5,
                                                    gamma=0.1)
    for epoch in range(num_epochs):
        train_one_epoch(model=model,
                        optimizer=optimizer, 
                        data_loader=data_loader,
                        device=device, 
                        epoch=epoch, 
                        print_freq=print_freq)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        lr_scheduler.step()
        evaluate(model=model, data_loader=data_loader_test, device=device)

In [51]:
HRSID_train = HRSIDSegmentationDataset(dataset_train, transforms=get_transform())
HRSID_test = HRSIDSegmentationDataset(dataset_test, transforms=get_transform())

In [52]:
x = np.array([0,0,0,255,255,0,0,0])
np.clip(x, 0, 1)

array([0, 0, 0, 1, 1, 0, 0, 0])

In [59]:
model = get_model(2)

In [60]:
do_training(model, HRSID_train, HRSID_test, num_epochs=20)


Using cuda
Epoch: [0]  [  0/911]  eta: 0:59:40  lr: 0.000002  loss: 5.1333 (5.1333)  loss_classifier: 0.5706 (0.5706)  loss_box_reg: 0.0896 (0.0896)  loss_mask: 2.2855 (2.2855)  loss_objectness: 1.9364 (1.9364)  loss_rpn_box_reg: 0.2512 (0.2512)  time: 3.9298  data: 3.4212  max mem: 4254
Epoch: [0]  [ 10/911]  eta: 0:11:59  lr: 0.000013  loss: 2.8339 (3.1338)  loss_classifier: 0.5309 (0.5410)  loss_box_reg: 0.1633 (0.1509)  loss_mask: 1.7565 (1.7786)  loss_objectness: 0.2123 (0.5874)  loss_rpn_box_reg: 0.0288 (0.0760)  time: 0.7989  data: 0.3470  max mem: 4437
Epoch: [0]  [ 20/911]  eta: 0:09:21  lr: 0.000024  loss: 2.7271 (3.2279)  loss_classifier: 0.5216 (0.5244)  loss_box_reg: 0.1254 (0.1370)  loss_mask: 1.6735 (1.7578)  loss_objectness: 0.1401 (0.7259)  loss_rpn_box_reg: 0.0264 (0.0829)  time: 0.4648  data: 0.0264  max mem: 4457
Epoch: [0]  [ 30/911]  eta: 0:08:24  lr: 0.000035  loss: 2.3342 (2.9703)  loss_classifier: 0.4812 (0.4997)  loss_box_reg: 0.1264 (0.1417)  loss_mask: 1.586

In [61]:
model_not_pretrained = get_model(2, False)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/k3vinli/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
23.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

53.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

83.4%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.io

In [62]:
do_training(model_not_pretrained, HRSID_train, HRSID_test, num_epochs=20)


Using cuda
Epoch: [0]  [  0/911]  eta: 1:08:48  lr: 0.000002  loss: 5.5138 (5.5138)  loss_classifier: 0.6979 (0.6979)  loss_box_reg: 0.0104 (0.0104)  loss_mask: 3.6724 (3.6724)  loss_objectness: 0.6959 (0.6959)  loss_rpn_box_reg: 0.4372 (0.4372)  time: 4.5324  data: 3.9458  max mem: 5719
Epoch: [0]  [ 10/911]  eta: 0:14:10  lr: 0.000013  loss: 7.0853 (6.7667)  loss_classifier: 0.7142 (0.7121)  loss_box_reg: 0.0334 (0.0312)  loss_mask: 5.6600 (5.1877)  loss_objectness: 0.6929 (0.6932)  loss_rpn_box_reg: 0.0756 (0.1426)  time: 0.9437  data: 0.3664  max mem: 5719
Epoch: [0]  [ 20/911]  eta: 0:11:00  lr: 0.000024  loss: 5.3873 (5.5885)  loss_classifier: 0.7104 (0.7100)  loss_box_reg: 0.0227 (0.0266)  loss_mask: 3.8680 (3.9986)  loss_objectness: 0.6932 (0.6938)  loss_rpn_box_reg: 0.0427 (0.1595)  time: 0.5518  data: 0.0105  max mem: 5719
Epoch: [0]  [ 30/911]  eta: 0:09:52  lr: 0.000035  loss: 3.5871 (4.8495)  loss_classifier: 0.6994 (0.7054)  loss_box_reg: 0.0183 (0.0255)  loss_mask: 2.045

In [ ]:
file_name = f"{datetime.now().strftime('%m_%d_%y, %H-%M-%S')}e15.pt"

In [ ]:
# https://pytorch.org/tutorials/beginner/saving_loading_models.html
# Saving Model
torch.save(model.state_dict(), os.path.abspath(f"maskrcnn_weights/{file_name}"))
torch.save(model.state_dict(), os.path.abspath(f"maskrcnn_weights/{file_name}"))